In [1]:
#pip install --force-reinstall agent_system/dist/mesa_restaurant_agents-0.0.1-py3-none-any.whl

Processing c:\users\s.neumann\documents\11_master_cas\code\mesa-restaurant-agents\agent_system\dist\mesa_restaurant_agents-0.0.1-py3-none-any.whl
  Using cached mesa-3.1.0-py3-none-any.whl.metadata (9.9 kB)
  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.2-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached scikit_learn-1.6.1-cp312-cp312-win_amd64.whl.metadata (15 kB)
  Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached plotly-5.24.1-py3-none-any.whl.metadata (7.3 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp312-cp312-win_amd64.whl.metadata (104 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
  Using 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.2 which is incompatible.


In [2]:
import os
import sys
import pandas as pd
import plotly.graph_objects as go
import itertools

import mesa
from mesa_restaurant_agents.model.restaurant_model import RestaurantModel
from mesa_restaurant_agents.visualization import *
from mesa_restaurant_agents.visualization import GridAnimator

# Parameter Values

In [3]:
grid_side_lengths = [5, 10, 15, 20, 30]
waiter_counts = [1, 2, 5, 10, 15]
days = [1, 2, 5, 15, 30]

# Functions

In [4]:
def run_model(run):
    steps = run['days']*144
    params = {"n_waiters":run['waiter_init_count'], "grid_width":run['grid_side_length'], "grid_height": run['grid_side_length']}

    results = mesa.batch_run(
        RestaurantModel,
        parameters=params,
        iterations=1,
        max_steps=steps,
        number_processes=1,
        data_collection_period=1,
        display_progress=True,
    )

    # df = pd.DataFrame(results)
    # numeric_cols = ['Step', 'Customer_Count', 'Average_Wait_Time', 'Average_Customer_Satisfaction', 'Revenue']
    # df[numeric_cols] = df[numeric_cols].fillna(0).astype(float)
    
    # non_empty_cols = [col for col in df.columns if df[col].notna().any()]
    # df[non_empty_cols].head()

    return results


def suppress_print(func):
    def wrapper(*args, **kwargs):
        original_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')
        result = func(*args, **kwargs)
        sys.stdout.close()
        sys.stdout = original_stdout
        return result
    return wrapper


run_model_print_suppressed = suppress_print(run_model)


def get_revenue_list(results):
    df = pd.DataFrame(results)
    daily_stats = df['Daily_Stats'].tolist()
    unique_daily_stats = []
    seen = set()
    for sublist in daily_stats:
        for item in sublist:
            item_tuple = tuple(item.items())
            if item_tuple not in seen:
                seen.add(item_tuple)
                unique_daily_stats.append(item)
    unique_daily_stats = [item for item in unique_daily_stats if item]
    daily_revenue = [stats['revenue'] for stats in unique_daily_stats]
    return daily_revenue


def print_analytics(daily_revenue, run):
    fig = go.Figure(data=[go.Bar(x=list(range(len(daily_revenue))), y=daily_revenue)])

    fig.update_layout(
        title=f'Daily Revenue for GridSize {run['grid_side_length']}, InitWaiters {run['waiter_init_count']} and Days {run['days']}',
        xaxis_title='Day',
        yaxis_title='Revenue (Money)'
    )
    fig.show()

def visualize_overview(overview):
    x_values = [f"Grid: {item['run']['grid_side_length']}, Waiters: {item['run']['waiter_init_count']}, Days: {item['run']['days']}" for item in overview]
    y_values = [item['mean_revenue'] for item in overview]

    fig = go.Figure(data=[go.Bar(x=x_values, y=y_values)])

    fig.update_layout(
        title="Mean Revenue by Run",
        xaxis_title="Run",
        yaxis_title="Mean Revenue",
        xaxis_tickangle=-45
    )

    fig.show()

# Visualization Example

In [5]:
results = run_model({"grid_side_length": 23, "waiter_init_count": 10, "days": 1})

  0%|          | 0/1 [00:00<?, ?it/s]

DEBUG: Waiter 3 picked up meat for customer 12
DEBUG: Waiter 3 step - available=True, carrying_food=1, target_pos=(15, 21)
Waiter 3 moved from (13, 13) to (14, 21), steps: 8
DEBUG: Waiter 3 served customer 12 - Order: meat, Price: $30.00
Waiter 3 moved from (14, 21) to (13, 13), steps: 8
DEBUG: Waiter 1 picked up vegetarian for customer 13
DEBUG: Waiter 1 picked up meat for customer 14
DEBUG: Waiter 1 step - available=True, carrying_food=2, target_pos=(13, 9)
Waiter 1 moved from (13, 13) to (12, 9), steps: 8
DEBUG: Waiter 1 served customer 13 - Order: vegetarian, Price: $20.00
DEBUG: Waiter 1 step - available=True, carrying_food=1, target_pos=(5, 9)
Waiter 1 moved from (12, 9) to (4, 9), steps: 8
DEBUG: Waiter 1 served customer 14 - Order: meat, Price: $30.00
Waiter 1 moved from (4, 9) to (12, 13), steps: 8
Waiter 1 moved from (12, 13) to (13, 13), steps: 1
DEBUG: Waiter 2 picked up vegetarian for customer 15
DEBUG: Waiter 2 picked up gluten_free for customer 16
DEBUG: Waiter 2 picked 

In [6]:
import pandas as pd
df = pd.DataFrame(results)
df
# df = df[df["RunId"] == 0]
# df['hours'] = df.apply(minutes_to_time, axis=1)

# data_grouped = df.groupby(['day','hours']).agg(
#     mean_customer_count=('Customer_Count', 'mean'),
#     mean_waiters_count=('Waiters_Count', 'mean'),
#     mean_waiting_time=('Average_Wait_Time', 'mean'),
#     mean_customer_satisfaction=('Average_Customer_Satisfaction', 'mean'),
#     mean_revenue=('Revenue', 'mean'),
#     mean_tips = ('Tips', 'mean')).reset_index()

# data_grouped

,RunId,iteration,Step,n_waiters,grid_width,grid_height,day,shift,time,Customer_Count,Waiters_Count,Average_Wait_Time,Average_Customer_Satisfaction,Revenue,Tips,Customer_Info,Waiter_Info,GridState,Daily_Stats
0,0,0,0,10,23,23,1,1.0,660,0,10,0.000000,100.000000,0.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
1,0,0,1,10,23,23,1,1.0,660,0,10,0.000000,100.000000,0.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
2,0,0,2,10,23,23,1,1.0,665,0,10,0.000000,100.000000,0.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
3,0,0,3,10,23,23,1,1.0,665,0,10,0.000000,100.000000,0.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
4,0,0,4,10,23,23,1,1.0,670,0,10,0.000000,100.000000,0.0,0.0,[],"[{'waiter_nr': 1, 'tips': 0, 'served_customers...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286,0,0,286,10,23,23,1,3.0,1375,11,4,61.818182,13.636364,7940.7,209.0,"[{'customer_nr': 563, 'waiting_time': 95, 'ord...","[{'waiter_nr': 1, 'tips': 46.30000000000002, '...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
287,0,0,287,10,23,23,1,3.0,1375,11,4,61.818182,13.636364,7940.7,209.0,"[{'customer_nr': 563, 'waiting_time': 95, 'ord...","[{'waiter_nr': 1, 'tips': 46.30000000000002, '...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
288,0,0,288,10,23,23,1,NaN,1380,9,4,61.666667,14.444444,7940.7,209.0,"[{'customer_nr': 564, 'waiting_time': 90, 'ord...","[{'waiter_nr': 1, 'tips': 46.30000000000002, '...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...",[{}]
289,0,0,289,10,23,23,2,1.0,660,0,7,0.000000,100.000000,0.0,209.0,[],"[{'waiter_nr': 1, 'tips': 46.30000000000002, '...","[{'pos': (13, 13), 'type': 'Kitchen'}, {'pos':...","[{'day': 1, 'customers_paid': 316, 'customers_..."


In [7]:
animator = GridAnimator(results)

ani = animator.animate_first_run()
ani.save('animated_heatmap.gif', writer='pillow', fps=2, dpi=80)

In [7]:
grouped_data = display_mean_step_results(results)

In [8]:
customer_infos_df = display_first_run_step_results_customer(results)
waiter_infos_df = display_first_run_step_results_waiter(results)

None


None


None


None


# Parameter Comparison

In [9]:
runs = [{"grid_side_length": gsl, "waiter_init_count": wc, "days": d} for gsl, wc, d in itertools.product(grid_side_lengths, waiter_counts, days)]
results_list = []
overview = []
best_run_counter = 0
best_run_revenue = 0
counter = 0
for run in runs:
    results = run_model_print_suppressed(run)
    results_list.append(results)
    daily_revenue = get_revenue_list(results)
    mean_revenue = sum(daily_revenue)/run['days']
    overview.append({"run": run, "mean_revenue": mean_revenue})
    if mean_revenue > best_run_revenue:
        best_run_counter = counter
        best_run_revenue = mean_revenue
    counter += 1

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
visualize_overview(overview)
best_results = results_list[best_run_counter]